In [1]:
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader, random_split
from gloss_dataset import GlossDataset
from gloss_model import GlossModel
from torch.utils.tensorboard import SummaryWriter


In [2]:
# get data for training
gd = GlossDataset()
input_size = gd[0][0].shape[1]
class_no = len(gd.classes)
input_size, class_no

(1596, 5)

In [3]:
# provide input and class size
model = GlossModel(input_size, class_no)
device = model.device
model.to(device)
optim = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()


In [4]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(device=device))

NVIDIA GeForce GTX 1650


In [5]:
# initialize sumamry writer
writer=SummaryWriter()

In [6]:
# Create testing and training dataLoader from single dataset using random_split
# and also set training epoch
split_ratio = 0.8
batch_size = 1
train_size = int(split_ratio*len(gd))
test_size = len(gd)-train_size
train_data, test_data = random_split(gd, [train_size, test_size])
train_dl = DataLoader(train_data, batch_size=batch_size,
                      shuffle=True, pin_memory=False)
test_dl = DataLoader(test_data, batch_size=batch_size,
                     shuffle=True, pin_memory=False)
epoch = 1000


In [7]:
# Start model training
model.train()
for i in range(epoch):
    total_loss=0
    for x_train, y_train in train_dl:
        xtrain, y_train = x_train.to(device), y_train.to(device)
        out = model(x_train)
        loss = loss_fn(out, y_train.argmax(dim=1))
        optim.zero_grad()
        loss.backward()
        optim.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_dl)
    writer.add_scalar("Loss/epoch", avg_loss, i)

    if (i % 10 == 0):
        print(f"Epoch {i+1}/{epoch}, Loss: {avg_loss:.4f}")

writer.close()

Epoch 1/1000, Loss: 1.6121
Epoch 11/1000, Loss: 1.6007
Epoch 21/1000, Loss: 1.5894
Epoch 31/1000, Loss: 1.5690
Epoch 41/1000, Loss: 1.5503
Epoch 51/1000, Loss: 1.5226
Epoch 61/1000, Loss: 1.4932
Epoch 71/1000, Loss: 1.4697
Epoch 81/1000, Loss: 1.4501
Epoch 91/1000, Loss: 1.4293
Epoch 101/1000, Loss: 1.4136
Epoch 111/1000, Loss: 1.3951
Epoch 121/1000, Loss: 1.3811
Epoch 131/1000, Loss: 1.3622
Epoch 141/1000, Loss: 1.3562
Epoch 151/1000, Loss: 1.3397
Epoch 161/1000, Loss: 1.3165
Epoch 171/1000, Loss: 1.3100
Epoch 181/1000, Loss: 1.3391
Epoch 191/1000, Loss: 1.2897
Epoch 201/1000, Loss: 1.2904
Epoch 211/1000, Loss: 1.2789
Epoch 221/1000, Loss: 1.2751
Epoch 231/1000, Loss: 1.3021
Epoch 241/1000, Loss: 1.2896
Epoch 251/1000, Loss: 1.2565
Epoch 261/1000, Loss: 1.2566
Epoch 271/1000, Loss: 1.2528
Epoch 281/1000, Loss: 1.2474
Epoch 291/1000, Loss: 1.2367
Epoch 301/1000, Loss: 1.2024
Epoch 311/1000, Loss: 1.2019
Epoch 321/1000, Loss: 1.1764
Epoch 331/1000, Loss: 1.1700
Epoch 341/1000, Loss: 1.1

In [8]:
# Evaluate Test set
model.eval()
tot_loss = 0
correct = 0
total = 0
with torch.no_grad():
    for x_test, y_test in test_dl:
        out = model(x_test)
        y_test=y_test.argmax(dim=1)
        loss = loss_fn(out, y_test)
        tot_loss += loss.item()
        y_pred = torch.argmax(out, dim=1)
        correct += torch.sum(torch.eq(y_pred, y_test)).item()
        total += y_test.shape[0]

acc = correct / total
loss = tot_loss / len(test_dl)
print(f"Test Acc {acc:.4f}\nTest Loss : {loss:.4f}\n")
print("Total test samples : ", total)
print("Correct predictions : ", correct)

Test Acc 0.7000
Test Loss : 1.2209

Total test samples :  20
Correct predictions :  14


In [9]:
# save the model
torch.save(model, "asl_recog_lstm.pt")
